# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

Pessoa(__id__, sexo, idade) \
Medicamento(__codigo__, nome) \
Uso_Medicamento(__id_pessoa__, __codigo_medicamento__, dias_de_uso)

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE Pessoa IF EXISTS;

CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/nhanes/demographic-person.csv');

In [3]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [4]:
DROP TABLE Medicamento IF EXISTS;
DROP TABLE Uso_Medicamento IF EXISTS;

CREATE TABLE Medicamento (
    Codigo VARCHAR(6),
    Nome VARCHAR(100),
    PRIMARY KEY(Codigo)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/nhanes/medications-drug.csv');

CREATE TABLE Uso_Medicamento (
    Id VARCHAR(5),
    Codigo_Medicamento VARCHAR(6),
    Dias_de_Uso INTEGER,
    FOREIGN KEY(Id)
        REFERENCES Pessoa(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Codigo_Medicamento)
        REFERENCES Medicamento(Codigo)
        
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/nhanes/medications-use.csv');

In [11]:
SELECT * FROM Medicamento;

In [12]:
SELECT * FROM Uso_Medicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [5]:
SELECT * FROM Pessoa WHERE Age > 60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [6]:
SELECT DISTINCT Age FROM Pessoa

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [7]:
SELECT P.Id, P.Age, U.Codigo_Medicamento, U.Dias_de_Uso FROM Pessoa P, Uso_Medicamento U WHERE P.Id = U.Id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [8]:
SELECT P.Id, P.Age, M.Nome, U.Dias_de_Uso FROM Pessoa P, Medicamento M, Uso_Medicamento U WHERE P.Id = U.Id AND M.Codigo = U.Codigo_Medicamento;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [9]:
SELECT DISTINCT M.Nome FROM Medicamento M, Uso_Medicamento U WHERE M.Codigo = U.Codigo_Medicamento;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [10]:
SELECT DISTINCT M.Nome FROM Medicamento M, Uso_Medicamento U WHERE M.Codigo = U.Codigo_Medicamento AND U.Dias_de_Uso > 2000;